In [45]:
import pandas as pd
import glob
import os

folder_path = "/Users/ravigurjar/Desktop/temp/sample/dessertation/data/raw"

# Get all CSV files in folder
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

df_list = []

for file in all_files:
    df = pd.read_excel(file)
    df_list.append(df)
    print(f"Loaded: {os.path.basename(file)}  Shape: {df.shape}")



Loaded: Dataset_2.xlsx  Shape: (153, 11)
Loaded: Dataset_3.xlsx  Shape: (153, 11)
Loaded: Dataset_10.xlsx  Shape: (153, 11)
Loaded: Dataset_8.xlsx  Shape: (153, 11)
Loaded: Dataset_4.xlsx  Shape: (153, 11)
Loaded: Dataset_5.xlsx  Shape: (153, 11)
Loaded: Dataset_11.xlsx  Shape: (153, 11)
Loaded: Dataset_9.xlsx  Shape: (153, 11)
Loaded: Dataset_12.xlsx  Shape: (153, 11)
Loaded: Dataset_6.xlsx  Shape: (153, 11)
Loaded: Dataset_7.xlsx  Shape: (153, 11)
Loaded: Dataset_1.xlsx  Shape: (153, 11)


In [49]:
merged_df = pd.concat(df_list, ignore_index=True)

In [51]:
import os

processed_folder = "/Users/ravigurjar/Desktop/temp/sample/dessertation/data/processed"
os.makedirs(processed_folder, exist_ok=True)

output_path = os.path.join(processed_folder, "merged_df.csv")

final_df.to_csv(output_path, index=False)
print("Saved successfully to:", output_path)

Saved successfully to: /Users/ravigurjar/Desktop/temp/sample/dessertation/data/processed/merged_df.csv


In [54]:
import pandas as pd
import requests

# 1. Load your Excel file
df = pd.read_csv("/Users/ravigurjar/Desktop/temp/sample/dessertation/data/processed/merged_df.csv").copy()

# ------------------------------------------------------------------
# 2. Parse Time → NASA date + hour
#    (adjust format=... if your Time string is different)
# ------------------------------------------------------------------
# This will handle common formats automatically (e.g. "2025-12-01 13:00:00" or "01-12-2025 13:00:00")
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# Date in NASA format YYYYMMDD (string)
df["nasa_date"] = df["Time"].dt.strftime("%Y%m%d")

# Hour as 2-digit string ("00"–"23")
df["hour_str"] = df["Time"].dt.strftime("%H")


In [56]:
import pandas as pd
import requests


# ------------------------------------------------------------------
# 2. Parse Time → nasa_date
# ------------------------------------------------------------------
# If Time is like "01-12-2025 13:00:00", you can force the format:
# df["Time"] = pd.to_datetime(df["Time"], format="%d-%m-%Y %H:%M:%S", errors="coerce")
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

df["nasa_date"] = df["Time"].dt.strftime("%Y%m%d")  # '20240503'

print("Sample of time parsing:")
print(df[["Time", "nasa_date"]].head())

# ------------------------------------------------------------------
# 3. NASA POWER DAILY API configuration
# ------------------------------------------------------------------
BASE_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

# Daily meteorological parameters you want
# (these are valid for daily AG; you can add/remove if needed)
PARAMS = "T2M,RH2M,WS10M,WD10M,PRECTOTCORR"

def fetch_nasa_daily(lat, lon, date_str):
    """
    Fetch DAILY NASA POWER meteorology for a single (lat, lon, date_str='YYYYMMDD').
    Returns dict like:
      {
        "T2M": {"20240503": 28.5, ...},
        "RH2M": {"20240503": 65.3, ...},
        ...
      }
    or None if it fails.
    """
    url = (
        f"{BASE_URL}?parameters={PARAMS}"
        f"&community=AG"
        f"&latitude={lat}&longitude={lon}"
        f"&start={date_str}&end={date_str}"
        f"&format=JSON"
    )

    try:
        r = requests.get(url, timeout=30)
    except Exception as e:
        print(f"Request error for ({lat}, {lon}, {date_str}): {e}")
        return None

    if r.status_code != 200:
        print(f"Bad status {r.status_code} for ({lat}, {lon}, {date_str})")
        try:
            print("Response:", r.json())
        except Exception:
            pass
        return None

    data = r.json()
    try:
        params_dict = data["properties"]["parameter"]
    except KeyError:
        print(f"No 'parameter' key in NASA response for ({lat}, {lon}, {date_str})")
        return None

    return params_dict


def extract_daily_value(param_dict, var_name, date_str):
    """
    From NASA DAILY parameter dict, extract value at given date.
    Keys look like: '20240503'
    """
    if param_dict is None or var_name not in param_dict:
        return None

    series = param_dict[var_name]   # dict of { "YYYYMMDD": value }
    return series.get(date_str, None)

# ------------------------------------------------------------------
# 4. Build a separate NASA DAILY dataframe (no merge)
# ------------------------------------------------------------------
meteo_vars = ["T2M", "RH2M", "WS10M", "WD10M", "PRECTOTCORR"]

# unique (Latitude, Longitude, nasa_date) points in your dataset
unique_points = df[["Latitude", "Longitude", "nasa_date"]].drop_duplicates()

# cache NASA responses per (lat, lon, date) so we don't call for every row
cache = {}  # (lat, lon, nasa_date) -> param_dict

rows = []   # will hold rows for nasa_df

print("Unique (Latitude, Longitude, nasa_date) rows:", len(unique_points))

for _, p in unique_points.iterrows():
    lat = p["Latitude"]
    lon = p["Longitude"]
    date_str = p["nasa_date"]

    cache_key = (lat, lon, date_str)

    if cache_key not in cache:
        print(f"Fetching DAILY NASA data for lat={lat}, lon={lon}, date={date_str} ...")
        cache[cache_key] = fetch_nasa_daily(lat, lon, date_str)

    param_dict = cache[cache_key]

    # build one output row for this (lat, lon, date)
    out = {
        "Latitude": lat,
        "Longitude": lon,
        "nasa_date": date_str,
    }

    for var in meteo_vars:
        out[var] = extract_daily_value(param_dict, var, date_str)

    rows.append(out)

# Create separate NASA meteorology dataframe
nasa_df = pd.DataFrame(rows)

print("NASA DAILY meteo dataframe preview:")
print(nasa_df.head())

# Optionally save it to file for later merging
# nasa_df.to_excel("/Users/ravigurjar/Desktop/temp/sample/dessertation/data/NASA_meteo_daily_only.xlsx", index=False)

Sample of time parsing:
                 Time nasa_date
0 2024-05-01 09:00:00  20240501
1 2024-05-01 05:00:00  20240501
2 2024-05-01 10:00:00  20240501
3 2020-11-19 16:00:00  20201119
4 2024-05-01 10:00:00  20240501
Unique (Latitude, Longitude, nasa_date) rows: 987
Fetching DAILY NASA data for lat=14.675886, lon=77.593027, date=20240501 ...
Fetching DAILY NASA data for lat=16.9872867, lon=81.7363176, date=20240501 ...
Fetching DAILY NASA data for lat=13.67, lon=79.35, date=20240501 ...
Fetching DAILY NASA data for lat=16.507014, lon=80.627767, date=20201119 ...
Fetching DAILY NASA data for lat=27.103358, lon=93.679645, date=20240501 ...
Fetching DAILY NASA data for lat=26.1875, lon=91.744194, date=20240501 ...
Fetching DAILY NASA data for lat=25.204762, lon=85.51496, date=20240501 ...
Fetching DAILY NASA data for lat=25.5626095, lon=84.663264, date=20240501 ...
Fetching DAILY NASA data for lat=25.444266, lon=86.140169, date=20231222 ...
Fetching DAILY NASA data for lat=26.80365, lon=84

In [57]:
merged = df.merge(
    nasa_df,
    on=["Latitude", "Longitude", "nasa_date"],
    how="left"
)

In [58]:
merged.head()

,City,AQI,PM2.5,PM10,O3,NO2,SO2,CO,Latitude,Longitude,Time,nasa_date,hour_str,T2M,RH2M,WS10M,WD10M,PRECTOTCORR
0,"Gulzarpet, Anantapur, India",75,75.0,61.0,6.8,4.7,3.6,7.2,14.675886,77.593027,2024-05-01 09:00:00,20240501,09,35.22,33.79,3.94,259.8,0.00
1,"Anand Kala Kshetram, Rajamahendravaram, India",19,5.0,19.0,6.5,1.1,4.7,5.8,16.987287,81.736318,2024-05-01 05:00:00,20240501,05,34.87,52.86,4.19,202.5,0.00
2,"Tirumala-APPCB, Tirupati, India",90,90.0,58.0,12.5,10.0,4.1,7.3,13.670000,79.350000,2024-05-01 10:00:00,20240501,10,35.10,46.06,4.31,176.7,0.00
3,"PWD Grounds, Vijayawada, India",-,52.0,NaN,5.1,0.7,4.9,4.2,16.507014,80.627767,2020-11-19 16:00:00,20201119,16,24.94,87.03,2.44,109.8,0.00
4,"Naharlagun, Naharlagun, India",127,127.0,57.0,1.0,1.1,1.7,2.8,27.103358,93.679645,2024-05-01 10:00:00,20240501,10,25.98,66.55,3.14,222.0,3.16


In [65]:
import pandas as pd

# 1. Load your dataset
df = merged.copy()

# ---------------------------------------------------
# 2. Clean column names (strip spaces)
#    (we keep original case because your columns are case-sensitive)
# ---------------------------------------------------
df.columns = df.columns.str.strip()

print("Original columns:", df.columns.tolist())

# ---------------------------------------------------
# 3. Detect and standardise timestamp column
# ---------------------------------------------------
time_candidates = ["timestamp", "Timestamp", "time", "Time", "TIME"]

time_col = None
for c in time_candidates:
    if c in df.columns:
        time_col = c
        break

if time_col is None:
    raise KeyError(
        f"No time/timestamp-like column found. Columns are: {df.columns.tolist()}"
    )

# Rename whatever we found to 'timestamp'
if time_col != "timestamp":
    df = df.rename(columns={time_col: "timestamp"})

# Convert to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# ---------------------------------------------------
# 4. Rename other columns if they exist
#    (we only rename if the old name is present)
# ---------------------------------------------------
rename_map = {
    "pm2.5": "pm25_ugm3",
    "PM2.5": "pm25_ugm3",
    "co": "co_ppm",
    "CO": "co_ppm",
    "no2": "no2_ppb",
    "NO2": "no2_ppb",
    "so2": "so2_ppb",
    "SO2": "so2_ppb",
    "o3": "o3_ppb",
    "O3": "o3_ppb",
    "t2m": "temperature_c",
    "T2M": "temperature_c",
    "rh2m": "relative_humidity",
    "RH2M": "relative_humidity",
    "ws10m": "wind_speed_ms",
    "WS10M": "wind_speed_ms",
    "wd10m": "wind_direction_deg",
    "WD10M": "wind_direction_deg",
    "aqi": "aqi_value",
    "AQI": "aqi_value",
    "Latitude": "latitude",
    "Longitude":"longitude",
}

for old, new in rename_map.items():
    if old in df.columns and new not in df.columns:
        df = df.rename(columns={old: new})

# ---------------------------------------------------
# 5. Ensure we have all final columns (create if missing)
# ---------------------------------------------------
desired_cols = [
    "timestamp",
    "latitude",
    "longitude",
    "pm25_ugm3",
    "co_ppm",
    "no2_ppb",
    "so2_ppb",
    "o3_ppb",
    "temperature_c",
    "relative_humidity",
    "wind_speed_ms",
    "wind_direction_deg",
    "aqi_value",
    "aqi_category",
    "binary_label",
    "calibration_factor",
]

for col in desired_cols:
    if col not in df.columns:
        df[col] = pd.NA

# ---------------------------------------------------
# 6. Reorder columns to the exact schema
# ---------------------------------------------------
df = df[desired_cols]




# Optional: save cleaned dataset
# df.to_excel(
#     "/Users/ravigurjar/Desktop/temp/sample/dessertation/data/final_aqi_dataset.xlsx",
#     index=False,
# )

Original columns: ['City', 'AQI', 'PM2.5', 'PM10', 'O3', 'NO2', 'SO2', 'CO', 'Latitude', 'Longitude', 'Time', 'nasa_date', 'hour_str', 'T2M', 'RH2M', 'WS10M', 'WD10M', 'PRECTOTCORR']


In [66]:
df.head()

,timestamp,latitude,longitude,pm25_ugm3,co_ppm,no2_ppb,so2_ppb,o3_ppb,temperature_c,relative_humidity,wind_speed_ms,wind_direction_deg,aqi_value,aqi_category,binary_label,calibration_factor
0,2024-05-01 09:00:00,14.675886,77.593027,75.0,7.2,4.7,3.6,6.8,35.22,33.79,3.94,259.8,75,<NA>,<NA>,<NA>
1,2024-05-01 05:00:00,16.987287,81.736318,5.0,5.8,1.1,4.7,6.5,34.87,52.86,4.19,202.5,19,<NA>,<NA>,<NA>
2,2024-05-01 10:00:00,13.670000,79.350000,90.0,7.3,10.0,4.1,12.5,35.10,46.06,4.31,176.7,90,<NA>,<NA>,<NA>
3,2020-11-19 16:00:00,16.507014,80.627767,52.0,4.2,0.7,4.9,5.1,24.94,87.03,2.44,109.8,-,<NA>,<NA>,<NA>
4,2024-05-01 10:00:00,27.103358,93.679645,127.0,2.8,1.1,1.7,1.0,25.98,66.55,3.14,222.0,127,<NA>,<NA>,<NA>


In [67]:
df = df.drop(columns=["aqi_category", "binary_label", "calibration_factor"])

In [68]:
df

,timestamp,latitude,longitude,pm25_ugm3,co_ppm,no2_ppb,so2_ppb,o3_ppb,temperature_c,relative_humidity,wind_speed_ms,wind_direction_deg,aqi_value
0,2024-05-01 09:00:00,14.675886,77.593027,75.0,7.2,4.7,3.6,6.8,35.22,33.79,3.94,259.8,75
1,2024-05-01 05:00:00,16.987287,81.736318,5.0,5.8,1.1,4.7,6.5,34.87,52.86,4.19,202.5,19
2,2024-05-01 10:00:00,13.670000,79.350000,90.0,7.3,10.0,4.1,12.5,35.10,46.06,4.31,176.7,90
3,2020-11-19 16:00:00,16.507014,80.627767,52.0,4.2,0.7,4.9,5.1,24.94,87.03,2.44,109.8,-
4,2024-05-01 10:00:00,27.103358,93.679645,127.0,2.8,1.1,1.7,1.0,25.98,66.55,3.14,222.0,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,2024-05-01 10:00:00,22.544808,88.340369,127.0,10.1,12.8,9.3,27.1,36.68,40.92,3.62,228.4,127
1832,2024-05-01 10:00:00,22.627847,88.380669,106.0,6.8,12.6,2.6,35.4,36.68,40.92,3.62,228.4,106
1833,2024-05-01 09:00:00,22.562630,88.363037,162.0,NaN,NaN,NaN,NaN,36.68,40.92,3.62,228.4,162
1834,2024-05-01 10:00:00,22.627847,88.380669,106.0,6.8,12.6,2.6,35.4,36.68,40.92,3.62,228.4,106


In [69]:
another_df = pd.read_csv("/Users/ravigurjar/Desktop/temp/sample/dessertation/data/processed/Air quality monitoring data.csv")

In [70]:
another_df

,timestamp,latitude,longitude,pm25_ugm3,co_ppm,no2_ppb,so2_ppb,o3_ppb,temperature_c,relative_humidity,wind_speed_ms,wind_direction_deg,aqi_value,aqi_category,binary_label,calibration_factor
0,01/01/2023 00:00,23.892448,90.393656,2.8,2.42,54.3,10.0,31.0,29.0,41.6,2.3,120.2,11,Good,Safe,0.978396
1,01/01/2023 01:00,23.815139,90.379102,36.9,3.38,51.7,6.2,57.5,23.9,42.7,1.1,51.2,104,Unhealthy for Sensitive Groups,Harmful,1.010800
2,01/01/2023 02:00,23.842481,90.417399,38.3,2.52,74.1,16.4,188.9,30.2,52.4,1.9,101.5,108,Unhealthy for Sensitive Groups,Harmful,0.951336
3,01/01/2023 03:00,23.808169,90.414357,30.8,2.25,89.4,21.8,166.1,37.2,46.2,2.4,323.7,90,Moderate,Safe,1.040067
4,01/01/2023 04:00,23.848632,90.370174,10.9,0.95,77.3,8.7,85.6,23.1,54.0,3.9,57.4,45,Good,Safe,0.933461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,09/07/2023 18:00,23.862992,90.445081,11.5,1.59,6.2,20.4,18.3,31.9,46.1,2.3,109.8,48,Good,Safe,1.053097
5995,09/07/2023 19:00,23.832901,90.440574,7.5,0.71,5.3,11.3,14.7,26.4,74.0,2.5,154.9,31,Good,Safe,1.056224
5996,09/07/2023 20:00,23.812617,90.365143,1.5,1.01,12.8,7.9,23.3,34.2,41.4,2.2,99.1,6,Good,Safe,0.982562
5997,09/07/2023 21:00,23.805677,90.374522,3.7,2.10,44.8,14.5,69.2,15.3,63.5,1.0,234.5,15,Good,Safe,1.097383


In [71]:
another_df = another_df.drop(columns=["aqi_category", "binary_label", "calibration_factor"])

In [72]:
another_df

,timestamp,latitude,longitude,pm25_ugm3,co_ppm,no2_ppb,so2_ppb,o3_ppb,temperature_c,relative_humidity,wind_speed_ms,wind_direction_deg,aqi_value
0,01/01/2023 00:00,23.892448,90.393656,2.8,2.42,54.3,10.0,31.0,29.0,41.6,2.3,120.2,11
1,01/01/2023 01:00,23.815139,90.379102,36.9,3.38,51.7,6.2,57.5,23.9,42.7,1.1,51.2,104
2,01/01/2023 02:00,23.842481,90.417399,38.3,2.52,74.1,16.4,188.9,30.2,52.4,1.9,101.5,108
3,01/01/2023 03:00,23.808169,90.414357,30.8,2.25,89.4,21.8,166.1,37.2,46.2,2.4,323.7,90
4,01/01/2023 04:00,23.848632,90.370174,10.9,0.95,77.3,8.7,85.6,23.1,54.0,3.9,57.4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,09/07/2023 18:00,23.862992,90.445081,11.5,1.59,6.2,20.4,18.3,31.9,46.1,2.3,109.8,48
5995,09/07/2023 19:00,23.832901,90.440574,7.5,0.71,5.3,11.3,14.7,26.4,74.0,2.5,154.9,31
5996,09/07/2023 20:00,23.812617,90.365143,1.5,1.01,12.8,7.9,23.3,34.2,41.4,2.2,99.1,6
5997,09/07/2023 21:00,23.805677,90.374522,3.7,2.10,44.8,14.5,69.2,15.3,63.5,1.0,234.5,15


In [73]:
final_df = pd.concat([df,another_df])

In [74]:
final_df

,timestamp,latitude,longitude,pm25_ugm3,co_ppm,no2_ppb,so2_ppb,o3_ppb,temperature_c,relative_humidity,wind_speed_ms,wind_direction_deg,aqi_value
0,2024-05-01 09:00:00,14.675886,77.593027,75.0,7.20,4.7,3.6,6.8,35.22,33.79,3.94,259.8,75
1,2024-05-01 05:00:00,16.987287,81.736318,5.0,5.80,1.1,4.7,6.5,34.87,52.86,4.19,202.5,19
2,2024-05-01 10:00:00,13.670000,79.350000,90.0,7.30,10.0,4.1,12.5,35.10,46.06,4.31,176.7,90
3,2020-11-19 16:00:00,16.507014,80.627767,52.0,4.20,0.7,4.9,5.1,24.94,87.03,2.44,109.8,-
4,2024-05-01 10:00:00,27.103358,93.679645,127.0,2.80,1.1,1.7,1.0,25.98,66.55,3.14,222.0,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,09/07/2023 18:00,23.862992,90.445081,11.5,1.59,6.2,20.4,18.3,31.90,46.10,2.30,109.8,48
5995,09/07/2023 19:00,23.832901,90.440574,7.5,0.71,5.3,11.3,14.7,26.40,74.00,2.50,154.9,31
5996,09/07/2023 20:00,23.812617,90.365143,1.5,1.01,12.8,7.9,23.3,34.20,41.40,2.20,99.1,6
5997,09/07/2023 21:00,23.805677,90.374522,3.7,2.10,44.8,14.5,69.2,15.30,63.50,1.00,234.5,15


In [79]:
# Convert to datetime
final_df["timestamp"] = pd.to_datetime(final_df["timestamp"], errors="coerce")

# Format consistently
final_df["timestamp"] = final_df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [80]:
import os

cleaned_folder = "/Users/ravigurjar/Desktop/temp/sample/dessertation/data/cleaned"
os.makedirs(cleaned_folder, exist_ok=True)

output_path = os.path.join(cleaned_folder, "final_df.csv")

final_df.to_csv(output_path, index=False)

print("Saved successfully to:", output_path)

Saved successfully to: /Users/ravigurjar/Desktop/temp/sample/dessertation/data/cleaned/final_df.csv
